### Finetuning Biomistral-7B Open-source LLM for an LLM-powered Question-answering agent

#### Install the required libraries

In [1]:
import torch

!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-pb2s_64c/unsloth_a2cc8d6912d147b7ae0ed4614dc5a61e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-pb2s_64c/unsloth_a2cc8d6912d147b7ae0ed4614dc5a61e
  Resolved https://github.com/unslothai/unsloth.git to commit dbba69b085b9d6049b57b48b882af7e9f29df5b2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 25.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 13.3 MB/s eta 0:00:00
  Using cached wheel-0.42.0-py3-none-any.whl.metadata (2.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install "git+https://github.com/huggingface/transformers.git"

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-5553fr62
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-5553fr62
  Resolved https://github.com/huggingface/transformers.git to commit 0ad770c3733f9478a8d9d0bc18cc6143877b47a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8593793 sha256=58134571d46f46fbde7f6d5d0ae9bb727b3dbf4f3773776842bf2c0943154428
  Stored in directory: /tmp/pip-ephem-wheel-cache-rtyn_zyk/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1

[notice] A new release of p

In [ ]:
!pip install torch==2.0.1

In [3]:
!pip install trl datasets -q


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


### Download 4-bit Biomistral-7B Model using Unsloth's FastLanguageModel class

In [4]:
from unsloth import FastLanguageModel
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "BioMistral/BioMistral-7B", # "unsloth/mistral-7b" for 16bit loading
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cu118)
    Python  3.10.13 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.2
   \\   /|    GPU: NVIDIA RTX A5000. Max memory: 23.679 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu118. CUDA = 8.6. CUDA Toolkit = 11.8.
\        /    Bfloat16 = TRUE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


pytorch_model.bin:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

## Add LoRA adapters

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length,
)

Unsloth 2024.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [6]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files='Train_.csv')

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
# Iterate through the dataset and print the first 5 rows
for i in range(5):
    print(dataset['train'][i])

{'ID': 'Q829', 'Instruction': 'Compare the laboratory confirmation methods for Chikungunya and diabetes, and which diseases are diagnosed through blood glucose measurements?', 'Response': 'Chikungunya is confirmed using serological tests and PCR, while diabetes diagnosis involves blood glucose measurements.', 'Reference Document': 'TG Booklet 6', 'Paragraph(s) Number': '154, 166', 'Keywords': 'Laboratory Confirmation For Chikungunya Vs. Diabetes'}
{'ID': 'Q721', 'Instruction': 'When should specimens be collected for Anthrax diagnosis, and how should they be prepared, stored, and transported?', 'Response': "Specimens should be collected during the vesicular stage, and caution is needed due to B. anthracis' high infectivity. Specimen collection methods and storage/transport details vary for different forms of Anthrax.", 'Reference Document': 'TG Booklet 6', 'Paragraph(s) Number': '140', 'Keywords': 'Anthrax Specimen Collection: Timing, Preparation, Transport'}
{'ID': 'Q464', 'Instruction

## Prepare Dataset for fine-tuning

In [8]:
alpaca_prompt = """Below is a question posed by healthcare professionals
including nurses, doctors, and researchers in Malawi,
all of whom actively engage in disease surveillance efforts.
Offer a response that is both accurate and concise, incorporating
relevant keywords to address the inquiry effectively.

### Question:
{}

### Answer:
{}

### Keywords:
{}
"""


def formatting_prompts_func(examples):
    questions = examples["Instruction"]
    answers = examples["Response"]
    keywords = examples["Keywords"]
    texts = []
    for question, answer, keyword in zip(questions, answers, keywords):
        text = alpaca_prompt.format(question, answer, keyword)
        texts.append(text)
    return { "text" : texts}

In [9]:
dataset_ = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/748 [00:00<?, ? examples/s]

## Training the Model

In [10]:
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim import AdamW


training_args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    num_train_epochs= 1,
    warmup_steps = 5,
    max_steps = 1080,
    learning_rate = 2e-4,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 1,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = "outputs",
)

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset_['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = training_args,
)

optimizer = AdamW(model.parameters(), lr=training_args.learning_rate)
total_steps = 1000
scheduler = CosineAnnealingLR(
    optimizer,
    T_max=total_steps,
    eta_min=0,  # Minimum learning rate
)

# Define a function to update the learning rate during training
def update_lr():
  scheduler.step()

Map:   0%|          | 0/748 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [11]:
gpu_stats = torch.cuda.get_device_properties(0)
# Get properties of the GPU device at index 0

start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
# Get the maximum reserved GPU memory in GB and round to 3 decimal places

max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
# Get the total GPU memory in GB and round to 3 decimal places

print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
# Display the GPU name and maximum memory

print(f"{start_gpu_memory} GB of memory reserved.")
# Display the reserved memory amount

GPU = NVIDIA RTX A5000. Max memory = 23.679 GB.
4.75 GB of memory reserved.


In [16]:
# trainer_stats = trainer.train()
# Training loop
for epoch in range(training_args.num_train_epochs):
    trainer_stats = trainer.train()
    update_lr()

NotImplementedError: No operator found for `memory_efficient_attention_forward` with inputs:
     query       : shape=(2, 168, 8, 4, 128) (torch.bfloat16)
     key         : shape=(2, 168, 8, 4, 128) (torch.bfloat16)
     value       : shape=(2, 168, 8, 4, 128) (torch.bfloat16)
     attn_bias   : <class 'xformers.ops.fmha.attn_bias.LowerTriangularMask'>
     p           : 0.0
`flshattF@0.0.0` is not supported because:
    xFormers wasn't build with CUDA support
    operator wasn't built - see `python -m xformers.info` for more info
`tritonflashattF` is not supported because:
    xFormers wasn't build with CUDA support
    operator wasn't built - see `python -m xformers.info` for more info
    operator does not support BMGHK format
    triton is not available
    Only work on pre-MLIR triton for now
`cutlassF` is not supported because:
    xFormers wasn't build with CUDA support
    operator wasn't built - see `python -m xformers.info` for more info
`smallkF` is not supported because:
 

In [22]:
model.save_pretrained("biomistral_lora_adapter_weights")

In [23]:
from huggingface_hub import login
login()

In [24]:
model.push_to_hub("Adeptschneider/biomistral_lora_adapter_weights")

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Adeptschneider/biomistral_lora_adapter_weights/commit/f30a3d387dbba662863c510b8e9fc973eb1d641f', commit_message='Upload model', commit_description='', oid='f30a3d387dbba662863c510b8e9fc973eb1d641f', pr_url=None, pr_revision=None, pr_num=None)

In [25]:
HUGGING_FACE_REPO_NAME="Adeptschneider/biomistral_lora_adapter_weights"

In [26]:
# Get peft config.
import transformers
from peft import PeftConfig
config = PeftConfig.from_pretrained(HUGGING_FACE_REPO_NAME)

# Get base model
model = transformers.AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                                          torch_dtype=torch.float16,      # GPTQ quantization requires fp16
                                                          return_dict=True)

# Load the Lora model.
from peft import PeftModel
model = PeftModel.from_pretrained(model, HUGGING_FACE_REPO_NAME)

adapter_config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

In [27]:
# Merge model and Lora adapter.
merged_model = model.merge_and_unload()

In [28]:
HUGGING_FACE_MERGED_REPO_NAME = 'biomistral-fine-tuned-model'
# Push to HF Hub.
merged_model.push_to_hub(HUGGING_FACE_MERGED_REPO_NAME)
tokenizer.push_to_hub(HUGGING_FACE_MERGED_REPO_NAME)

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Adeptschneider/biomistral-fine-tuned-model/commit/426b0cbb9ebc4943711113c414dfe05775ea7540', commit_message='Upload tokenizer', commit_description='', oid='426b0cbb9ebc4943711113c414dfe05775ea7540', pr_url=None, pr_revision=None, pr_num=None)

In [10]:
alpaca_prompt_ = """Below is an instruction that describes a task.
Write a response that appropriately completes the request and the
reference document where information in the response was retrieved,
paragraph number, and the keywords in the response.

### Instruction:
{}
"""

In [11]:
from transformers import pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, device_map='auto', batch_size=8)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

In [12]:
import pandas as pd
test_df = pd.read_csv('./Test.csv')

In [13]:
test_df.head()

,ID,Question Text
0,Q4,"What is the definition of ""unusual event"""
1,Q5,What is Community Based Surveillance (CBS)?
2,Q9,What kind of training should members of VHC re...
3,Q10,What is indicator based surveillance (IBS)?
4,Q13,What is Case based surveillance?


In [14]:
submission_df = pd.DataFrame(columns=['ID', 'Target'])

In [15]:
submission_df.head()

,ID,Target


In [16]:
import re

def extract_info(response):
    # Initialize variables to store extracted information
    instruction = ""
    response_text = ""
    reference_document = ""
    paragraph_number = ""
    keywords = []

    # Define patterns for extracting information
    instruction_pattern = r"### Instruction:\n(.*?)\n\n"
    response_pattern = r"### Response:\n(.*?)\n\n"
    reference_pattern = r"### Reference Document:\n(.*?)\n\n"
    paragraph_pattern = r"### Paragraph\(s\) Number:\n(.*?)\n\n"
    keywords_pattern = r"### Keywords:\n(.*?)\n\n"

    # Extract information using regular expressions
    instruction_match = re.search(instruction_pattern, response)
    response_match = re.search(response_pattern, response)
    reference_match = re.search(reference_pattern, response)
    paragraph_match = re.search(paragraph_pattern, response)
    keywords_match = re.search(keywords_pattern, response)

    # Check if matches were found and extract information
    if instruction_match:
        instruction = instruction_match.group(1)
    if response_match:
        response_text = response_match.group(1)
    if reference_match:
        reference_document = reference_match.group(1)
    if paragraph_match:
        paragraph_number = paragraph_match.group(1)
    if keywords_match:
        keywords = keywords_match.group(1).strip()

    # Return the extracted information
    return instruction, response_text, reference_document, paragraph_number, keywords


In [17]:
from tqdm import tqdm

for index, row in tqdm(test_df.iterrows(), total=len(test_df)):
    prompt = alpaca_prompt_.format(row['Question Text'],)
    result = pipe(prompt)
    instruction, response_text, reference_document, paragraph_number, keywords = extract_info(result[0]['generated_text'])

    submission_df = pd.concat([
        submission_df,
        pd.DataFrame([{
            "ID": f"{row['ID']}_question_answer",
            "Target": response_text
        }, {
            "ID": f"{row['ID']}_keywords",
            "Target": keywords
        }, {
            "ID": f"{row['ID']}_paragraph(s)_number",
            "Target": paragraph_number
        }, {
            "ID": f"{row['ID']}_reference_document",
            "Target": reference_document
        }])
    ], ignore_index=True)


  2%|▏         | 10/499 [02:03<1:38:52, 12.13s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100%|██████████| 499/499 [1:39:31<00:00, 11.97s/it]


In [18]:
submission_df.head()

,ID,Target
0,Q4_question_answer,An unusual event is any event that does not fi...
1,Q4_keywords,[]
2,Q4_paragraph(s)_number,935
3,Q4_reference_document,TG Booklet 2
4,Q5_question_answer,Community-based surveillance (CBS) is a type o...


In [19]:
submission_df.to_csv("submission__.csv", index=False)